# Simple functions to showcase automated file transfer to nextcloud

## Functions to create dummy CML data CSV files

In [1]:
import pandas as pd
import xarray as xr
import datetime

ds_cmls = xr.open_dataset('openMRG_CML_180minutes.nc')
# cut to exactly 60 minutes
ds_cmls = ds_cmls.sel(time=slice('2015-08-27T01:00:00.00', '2015-08-27T01:59:00.00'))

def create_cml_data_for_current_minute():
    t_now = datetime.datetime.utcnow()
    df = (
        ds_cmls
        .isel(time=(ds_cmls.time.dt.minute == t_now.minute))
        .isel(sublink_id=0)
        .to_dataframe()
    )
    return df.loc[:, ['tsl', 'rsl']]

def write_cml_data_to_file(df, data_dir='cml_data', print_info=False):
    import os
    
    os.makedirs(data_dir, exist_ok = True)
    t_now = datetime.datetime.utcnow()
    fn = f'{data_dir}/{t_now:%Y%m%d_%H%M%S}_cml_dummy_data.csv'
    df.to_csv(fn)
    if print_info:
        print(f'created {fn}')


create_cml_data_for_current_minute().head()

,,tsl,rsl
cml_id,time,,
10001,2015-08-27 01:34:00,1.0,-46.400002
10002,2015-08-27 01:34:00,0.0,-41.000000
10003,2015-08-27 01:34:00,-5.0,-39.799999
10004,2015-08-27 01:34:00,-1.0,-49.500000
10005,2015-08-27 01:34:00,4.0,-45.700001


In [2]:
write_cml_data_to_file(create_cml_data_for_current_minute(), print_info=True)

created cml_data/20241002_203437_cml_dummy_data.csv


## Functions to move files to nextcloud

In [6]:
import requests
import base64

def upload_file_to_nextcloud(file_path, username, password, nextcloud_url='https://nextcloud.imk-ifu.kit.edu'):
    """Uploads a file to Nextcloud using the WebDAV API.

    Args:
        file_path: The path to the file to upload.
        nextcloud_url: The URL of the Nextcloud instance.
        username: The Nextcloud username.
        password: The Nextcloud password.

    Returns:
        True if the upload was successful, False otherwise.
    """

    import os
    fn = os.path.split(file_path)[-1]
    url = f"{nextcloud_url}/public.php/webdav/{fn}"
    
    with open(file_path, "rb") as f:
        response = requests.put(
            url, 
            auth=('m4rx7xLHzHbHSm3', ''), 
            headers = {'X-Requested-With': 'XMLHttpRequest',},
            data=f,
        )

    if response.status_code == 201:
        print(f"File {file_path} uploaded successfully!")
        return True
    else:
        print(f"Error uploading {file_path}: {response.status_code}")
        return False

def move_files_to_nextcloud(data_dir):
    import glob
    import os
    fn_list = glob.glob(f'{data_dir}/*.csv')
    for fn in fn_list:
        if upload_file_to_nextcloud(fn, username='not_yet_used', password='not_yet_used'):
            # Delete the local file if the upload was successful
            os.remove(fn)
            print(f"File {fn} deleted locally.")


move_files_to_nextcloud(data_dir='cml_data')

File cml_data/20241002_202328_cml_dummy_data.csv uploaded successfully!
File cml_data/20241002_202328_cml_dummy_data.csv deleted locally.
File cml_data/20241002_202253_cml_dummy_data.csv uploaded successfully!
File cml_data/20241002_202253_cml_dummy_data.csv deleted locally.
File cml_data/20241002_202308_cml_dummy_data.csv uploaded successfully!
File cml_data/20241002_202308_cml_dummy_data.csv deleted locally.
File cml_data/20241002_202243_cml_dummy_data.csv uploaded successfully!
File cml_data/20241002_202243_cml_dummy_data.csv deleted locally.
File cml_data/20241002_202318_cml_dummy_data.csv uploaded successfully!
File cml_data/20241002_202318_cml_dummy_data.csv deleted locally.
File cml_data/20241002_202238_cml_dummy_data.csv uploaded successfully!
File cml_data/20241002_202238_cml_dummy_data.csv deleted locally.
File cml_data/20241002_202303_cml_dummy_data.csv uploaded successfully!
File cml_data/20241002_202303_cml_dummy_data.csv deleted locally.
File cml_data/20241002_202258_cml_

## Loop to create files, run in the background


In [7]:
import time
import multiprocessing as mp

def file_creation_loop():
    for i in range(20):
        write_cml_data_to_file(create_cml_data_for_current_minute(), print_info=True)
        time.sleep(5)

# required on MacOS, see https://github.com/uqfoundation/multiprocess/issues/65
mp.set_start_method('fork', force=True) 

mp.Process(target=file_creation_loop).start()

created cml_data/20241002_203549_cml_dummy_data.csv
created cml_data/20241002_203554_cml_dummy_data.csv
created cml_data/20241002_203559_cml_dummy_data.csv
created cml_data/20241002_203604_cml_dummy_data.csv
created cml_data/20241002_203609_cml_dummy_data.csv
created cml_data/20241002_203614_cml_dummy_data.csv
created cml_data/20241002_203619_cml_dummy_data.csv
created cml_data/20241002_203624_cml_dummy_data.csv
created cml_data/20241002_203629_cml_dummy_data.csv
created cml_data/20241002_203634_cml_dummy_data.csv
created cml_data/20241002_203639_cml_dummy_data.csv
created cml_data/20241002_203644_cml_dummy_data.csv
created cml_data/20241002_203649_cml_dummy_data.csv
created cml_data/20241002_203654_cml_dummy_data.csv
created cml_data/20241002_203700_cml_dummy_data.csv
created cml_data/20241002_203705_cml_dummy_data.csv
created cml_data/20241002_203710_cml_dummy_data.csv
created cml_data/20241002_203715_cml_dummy_data.csv
created cml_data/20241002_203720_cml_dummy_data.csv
created cml_

In [9]:
move_files_to_nextcloud(data_dir='cml_data')

File cml_data/20241002_203604_cml_dummy_data.csv uploaded successfully!
File cml_data/20241002_203604_cml_dummy_data.csv deleted locally.
File cml_data/20241002_203609_cml_dummy_data.csv uploaded successfully!
File cml_data/20241002_203609_cml_dummy_data.csv deleted locally.
File cml_data/20241002_203619_cml_dummy_data.csv uploaded successfully!
File cml_data/20241002_203619_cml_dummy_data.csv deleted locally.
File cml_data/20241002_203614_cml_dummy_data.csv uploaded successfully!
File cml_data/20241002_203614_cml_dummy_data.csv deleted locally.
File cml_data/20241002_203629_cml_dummy_data.csv uploaded successfully!
File cml_data/20241002_203629_cml_dummy_data.csv deleted locally.
File cml_data/20241002_203624_cml_dummy_data.csv uploaded successfully!
File cml_data/20241002_203624_cml_dummy_data.csv deleted locally.
File cml_data/20241002_203634_cml_dummy_data.csv uploaded successfully!
File cml_data/20241002_203634_cml_dummy_data.csv deleted locally.
File cml_data/20241002_203639_cml_